In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR,LinearSVR
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from glob import glob
from copy import deepcopy
import lightgbm as lgb
# import keras
import tensorflow as tf
from tqdm.auto import tqdm
import optuna
from scipy.optimize import minimize,minimize_scalar
# import h2o
# from h2o.automl import H2OAutoML
# h2o.init()

In [2]:
train = pd.read_json('../input/stanford-covid-vaccine/train.json',lines=True).drop('index',axis=1)
label = np.load('../input/covid19fe/label.npy')
# SN_filter = np.where((train.SN_filter == 1).values)[0]
SN_filter = np.where((train.SN_filter == 1).values & (train.signal_to_noise>1))[0]

In [3]:
y1 = label[SN_filter,:68,0].reshape(-1)
y2 = label[SN_filter,:68,1].reshape(-1)
y3 = label[SN_filter,:68,2].reshape(-1)

In [4]:
oofs = []
paths = sorted(glob("../input/ensemble-predictions/*.npy"))[:-2]

for path in paths:
    oofs.append(np.load(path))
    if len(oofs[-1]) == 2400:
        oofs[-1] = oofs[-1][SN_filter]
    elif len(oofs[-1]) == 1589:
        oofs[-1] = oofs[-1][train[train.SN_filter == 1].signal_to_noise>1]
paths

['../input/ensemble-predictions/oof_0.19768.npy',
 '../input/ensemble-predictions/oof_0.19798.npy',
 '../input/ensemble-predictions/oof_0.19868.npy',
 '../input/ensemble-predictions/oof_0.19886.npy',
 '../input/ensemble-predictions/oof_0.19926.npy',
 '../input/ensemble-predictions/oof_0.19936.npy',
 '../input/ensemble-predictions/oof_0.19949.npy',
 '../input/ensemble-predictions/oof_0.19983.npy',
 '../input/ensemble-predictions/oof_0.19985.npy',
 '../input/ensemble-predictions/oof_0.19999.npy',
 '../input/ensemble-predictions/oof_0.20047.npy',
 '../input/ensemble-predictions/oof_0.20053.npy',
 '../input/ensemble-predictions/oof_0.20075.npy',
 '../input/ensemble-predictions/oof_0.20087.npy',
 '../input/ensemble-predictions/oof_0.20092.npy',
 '../input/ensemble-predictions/oof_0.20117.npy',
 '../input/ensemble-predictions/oof_0.20145.npy',
 '../input/ensemble-predictions/oof_0.20283.npy',
 '../input/ensemble-predictions/oof_0.20305.npy',
 '../input/ensemble-predictions/oof_0.20328.npy',


In [5]:
X1 = np.concatenate([x[:,:,0].reshape(-1,1) for x in oofs],axis=-1)
X2 = np.concatenate([x[:,:,1].reshape(-1,1) for x in oofs],axis=-1)
X3 = np.concatenate([x[:,:,2].reshape(-1,1) for x in oofs],axis=-1)

In [6]:
def feature_selection(X,y,useindex):
#     clf = LinearRegression(fit_intercept = False)
    clf = Ridge(alpha=80,fit_intercept = False,normalize=True)
    idx = []
    for j in range(X.shape[-1]):
        if j in useindex:
            idx.append(j)
    base_score = -cross_val_score(clf, X[:,idx], y, cv=5,scoring = 'neg_root_mean_squared_error').mean()
    score = []
    for i in range(X.shape[-1]):
        if i not in useindex:
            score.append(99999)
            continue
        idx = []
        for j in range(X.shape[-1]):
            if j in useindex:
                if j != i:
                    idx.append(j)
        cv_score = -cross_val_score(clf, X[:,idx], y, cv=5,scoring = 'neg_root_mean_squared_error').mean()
        score.append(cv_score)
    score = np.array(score)
    if score.min() < base_score:
        i = score.argmin()
        print(i,score.min())
        idx = []
        for j in range(X.shape[-1]):
            if j in useindex:
                if j != i:
                    idx.append(j)
        return feature_selection(X,y,idx)
    else:
        return useindex

In [7]:
select_col1 = feature_selection(X1,y1,np.arange(X1.shape[-1]))

19 0.170607020303081
22 0.17059687422893752
14 0.17058796289744574
29 0.1705808122532694
2 0.17057484267278675
13 0.17056975807928582
12 0.17056404161432953
20 0.17055938022509315
23 0.17055589497430163
15 0.17055199058513937
21 0.17054730148401373
27 0.1705434203925836
6 0.17054096305800562
28 0.17053983312049675


In [8]:
select_col2 = feature_selection(X2,y2,np.arange(X2.shape[-1]))

15 0.2160290456130412
27 0.21601239123539603
21 0.2159909830522186
6 0.2159775059683736
29 0.2159639903024519
11 0.21595150672816849
13 0.2159450650514892
23 0.21594088349984783
24 0.21594033284464142
20 0.21594021031817193
12 0.21593916666737506


In [9]:
select_col3 = feature_selection(X3,y3,np.arange(X3.shape[-1]))

13 0.1748560955711294
32 0.1748374061356097
20 0.17482308130172602
31 0.17480968620346388
17 0.1747983129494345
21 0.17479077401345172
24 0.1747847000929839
5 0.17477997742942514
26 0.17477855517605892
15 0.17477784250159295
25 0.1747775692136326


In [10]:
res = minimize(lambda x: np.sqrt(cross_val_score(Ridge(alpha=x,fit_intercept = False,normalize=True), X1[:,select_col1], y1, cv=5,scoring = lambda estimator,x,y: ((estimator.predict(x)-y)**2).mean()).mean())
, [100], method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
clf1 = Ridge(alpha=res.x,fit_intercept = False,normalize=True)

# res = minimize_scalar(lambda x: np.sqrt(cross_val_score(LinearSVR(C=x,fit_intercept=False,dual=False,loss='squared_epsilon_insensitive'), X1[:,select_col1], y1, cv=5,scoring = lambda estimator,x,y: ((estimator.predict(x)-y)**2).mean()).mean())
# , [1.0], method='Bounded',bounds=(0.0,1e3),options={'xatol': 1e-8, 'disp': True})
# clf1 = LinearSVR(C=res.x,fit_intercept=False,dual=False,loss='squared_epsilon_insensitive')

res.x

Optimization terminated successfully.
         Current function value: 0.170567
         Iterations: 30
         Function evaluations: 72


array([102.88025856])

In [11]:
clf1.fit(X1[:,select_col1],y1)
y1_pred = clf1.predict(X1[:,select_col1])
l1_weights = clf1.coef_
l1_intercept = clf1.intercept_
l1_weights,l1_intercept

(array([ 0.07889855,  0.12394121,  0.09904248,  0.04140526,  0.04131426,
         0.08080825,  0.07182798,  0.08400228,  0.08865465,  0.0761471 ,
         0.07470103,  0.06886259,  0.04044809,  0.04563309, -0.02170371,
         0.0673007 , -0.06755744,  0.04933872, -0.0270002 ]),
 0.0)

In [12]:
res = minimize(lambda x: np.sqrt(cross_val_score(Ridge(alpha=x,fit_intercept = False,normalize=True), X2[:,select_col2], y2, cv=5,scoring = lambda estimator,x,y: ((estimator.predict(x)-y)**2).mean()).mean())
, [100], method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
clf2 = Ridge(alpha=res.x,fit_intercept = False,normalize=True)

# res = minimize_scalar(lambda x: np.sqrt(cross_val_score(LinearSVR(C=x,fit_intercept=False,dual=False,loss='squared_epsilon_insensitive'), X2[:,select_col2], y2, cv=5,scoring = lambda estimator,x,y: ((estimator.predict(x)-y)**2).mean()).mean())
# , [1.0], method='Bounded',bounds=(0.0,1e3),options={'xatol': 1e-8, 'disp': True})
# clf2 = LinearSVR(C=res.x,fit_intercept=False,dual=False,loss='squared_epsilon_insensitive')

res.x

Optimization terminated successfully.
         Current function value: 0.216046
         Iterations: 32
         Function evaluations: 72


array([88.06108475])

In [13]:
clf2.fit(X2[:,select_col2],y2)
y2_pred = clf2.predict(X2[:,select_col2])
l2_weights = clf2.coef_
l2_intercept = clf2.intercept_
l2_weights,l2_intercept

(array([ 0.09624703,  0.12979714,  0.04880182,  0.06489577,  0.06648062,
         0.04668762,  0.11819444,  0.08209066,  0.0965125 ,  0.14590913,
        -0.03109661,  0.09648562,  0.03851601,  0.05760527,  0.03439749,
         0.05116822, -0.04507748,  0.03436544, -0.02607245, -0.07528783,
         0.03450136, -0.04040005]),
 0.0)

In [14]:
res = minimize(lambda x: np.sqrt(cross_val_score(Ridge(alpha=x,fit_intercept = False,normalize=True), X3[:,select_col3], y3, cv=5,scoring = lambda estimator,x,y: ((estimator.predict(x)-y)**2).mean()).mean())
, [100], method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
clf3 = Ridge(alpha=res.x,fit_intercept = False,normalize=True)

# res = minimize_scalar(lambda x: np.sqrt(cross_val_score(LinearSVR(C=x,fit_intercept=False,dual=False,loss='squared_epsilon_insensitive'), X3[:,select_col3], y3, cv=5,scoring = lambda estimator,x,y: ((estimator.predict(x)-y)**2).mean()).mean())
# , [1.0], method='Bounded',bounds=(0.0,1e3),options={'xatol': 1e-8, 'disp': True})
# clf3 = LinearSVR(C=res.x,fit_intercept=False,dual=False,loss='squared_epsilon_insensitive')

res.x

Optimization terminated successfully.
         Current function value: 0.174870
         Iterations: 36
         Function evaluations: 82


array([53.99352789])

In [15]:
clf3.fit(X3[:,select_col3],y3)
y3_pred = clf3.predict(X3[:,select_col3])
l3_weights = clf3.coef_
l3_intercept = clf3.intercept_
l3_weights,l3_intercept

(array([ 0.11682189,  0.15391301,  0.04828973,  0.10971558,  0.07034351,
         0.04834893,  0.11166402,  0.06448628,  0.09155653,  0.13718822,
         0.02184994,  0.03321369, -0.05974202,  0.07816287,  0.09002974,
         0.08739725,  0.01525135, -0.03490106, -0.03275975, -0.03843358,
        -0.03137855, -0.05951804]),
 0.0)

In [16]:
l1_score = np.sqrt(np.mean((y1_pred-y1)**2))
l1_cv_score = np.sqrt(cross_val_score(clf1, X1[:,select_col1], y1, cv=5,scoring = lambda estimator,x,y: ((estimator.predict(x)-y)**2).mean()).mean())
l1_score,l1_cv_score

(0.17036453700582446, 0.17056713993445663)

In [17]:
l2_score = np.sqrt(np.mean((y2_pred-y2)**2))
l2_cv_score = np.sqrt(cross_val_score(clf2, X2[:,select_col2], y2, cv=5,scoring = lambda estimator,x,y: ((estimator.predict(x)-y)**2).mean()).mean())
l2_score,l2_cv_score

(0.21579818817510663, 0.2160462590788587)

In [18]:
l3_score = np.sqrt(np.mean((y3_pred-y3)**2))
l3_cv_score = np.sqrt(cross_val_score(clf3, X3[:,select_col3], y3, cv=5,scoring = lambda estimator,x,y: ((estimator.predict(x)-y)**2).mean()).mean())
l3_score,l3_cv_score

(0.17465531836878437, 0.17486954321789988)

In [19]:
(l1_score+l2_score+l3_score)/3,(l1_cv_score+l2_cv_score+l3_cv_score)/3 

(0.18693934784990515, 0.1871609807437384)

In [20]:
ss = pd.read_csv("../input/stanford-covid-vaccine/sample_submission.csv").set_index('id_seqpos')

In [21]:
from glob import glob
sub = []
paths = sorted(glob("../input/ensemble-predictions/*csv"))

for path in paths:
    df = pd.read_csv(path)
    df = df.set_index('id_seqpos')[['reactivity','deg_Mg_pH10','deg_pH10','deg_Mg_50C','deg_50C']].loc[ss.index]
    sub.append(df)
paths

['../input/ensemble-predictions/submission_cnn_0.19768.csv',
 '../input/ensemble-predictions/submission_cnn_0.19798.csv',
 '../input/ensemble-predictions/submission_cnn_0.19868.csv',
 '../input/ensemble-predictions/submission_cnn_0.19886.csv',
 '../input/ensemble-predictions/submission_cnn_0.19926.csv',
 '../input/ensemble-predictions/submission_cnn_0.19936.csv',
 '../input/ensemble-predictions/submission_cnn_0.19949.csv',
 '../input/ensemble-predictions/submission_cnn_0.19983.csv',
 '../input/ensemble-predictions/submission_cnn_0.19985.csv',
 '../input/ensemble-predictions/submission_cnn_0.19999.csv',
 '../input/ensemble-predictions/submission_cnn_0.20047.csv',
 '../input/ensemble-predictions/submission_cnn_0.20053.csv',
 '../input/ensemble-predictions/submission_cnn_0.20075.csv',
 '../input/ensemble-predictions/submission_cnn_0.20087.csv',
 '../input/ensemble-predictions/submission_cnn_0.20092.csv',
 '../input/ensemble-predictions/submission_cnn_0.20117.csv',
 '../input/ensemble-pred

In [22]:
np.array(paths)[select_col1]

array(['../input/ensemble-predictions/submission_cnn_0.19768.csv',
       '../input/ensemble-predictions/submission_cnn_0.19798.csv',
       '../input/ensemble-predictions/submission_cnn_0.19886.csv',
       '../input/ensemble-predictions/submission_cnn_0.19926.csv',
       '../input/ensemble-predictions/submission_cnn_0.19936.csv',
       '../input/ensemble-predictions/submission_cnn_0.19983.csv',
       '../input/ensemble-predictions/submission_cnn_0.19985.csv',
       '../input/ensemble-predictions/submission_cnn_0.19999.csv',
       '../input/ensemble-predictions/submission_cnn_0.20047.csv',
       '../input/ensemble-predictions/submission_cnn_0.20053.csv',
       '../input/ensemble-predictions/submission_cnn_0.20145.csv',
       '../input/ensemble-predictions/submission_cnn_0.20283.csv',
       '../input/ensemble-predictions/submission_cnn_0.20305.csv',
       '../input/ensemble-predictions/submission_cnn_0.20423.csv',
       '../input/ensemble-predictions/submission_cnn_0.20525.c

In [23]:
np.array(paths)[select_col2]

array(['../input/ensemble-predictions/submission_cnn_0.19768.csv',
       '../input/ensemble-predictions/submission_cnn_0.19798.csv',
       '../input/ensemble-predictions/submission_cnn_0.19868.csv',
       '../input/ensemble-predictions/submission_cnn_0.19886.csv',
       '../input/ensemble-predictions/submission_cnn_0.19926.csv',
       '../input/ensemble-predictions/submission_cnn_0.19936.csv',
       '../input/ensemble-predictions/submission_cnn_0.19983.csv',
       '../input/ensemble-predictions/submission_cnn_0.19985.csv',
       '../input/ensemble-predictions/submission_cnn_0.19999.csv',
       '../input/ensemble-predictions/submission_cnn_0.20047.csv',
       '../input/ensemble-predictions/submission_cnn_0.20092.csv',
       '../input/ensemble-predictions/submission_cnn_0.20145.csv',
       '../input/ensemble-predictions/submission_cnn_0.20283.csv',
       '../input/ensemble-predictions/submission_cnn_0.20305.csv',
       '../input/ensemble-predictions/submission_cnn_0.20328.c

In [24]:
np.array(paths)[select_col3]

array(['../input/ensemble-predictions/submission_cnn_0.19768.csv',
       '../input/ensemble-predictions/submission_cnn_0.19798.csv',
       '../input/ensemble-predictions/submission_cnn_0.19868.csv',
       '../input/ensemble-predictions/submission_cnn_0.19886.csv',
       '../input/ensemble-predictions/submission_cnn_0.19926.csv',
       '../input/ensemble-predictions/submission_cnn_0.19949.csv',
       '../input/ensemble-predictions/submission_cnn_0.19983.csv',
       '../input/ensemble-predictions/submission_cnn_0.19985.csv',
       '../input/ensemble-predictions/submission_cnn_0.19999.csv',
       '../input/ensemble-predictions/submission_cnn_0.20047.csv',
       '../input/ensemble-predictions/submission_cnn_0.20053.csv',
       '../input/ensemble-predictions/submission_cnn_0.20075.csv',
       '../input/ensemble-predictions/submission_cnn_0.20092.csv',
       '../input/ensemble-predictions/submission_cnn_0.20145.csv',
       '../input/ensemble-predictions/submission_cnn_0.20305.c

In [25]:
x = l1_intercept
for i,col in enumerate(select_col1):
    x += l1_weights[i] * sub[col]['reactivity'].values
ss['reactivity'] = x

In [26]:
x = l2_intercept
for i,col in enumerate(select_col2):
    x += l2_weights[i] * sub[col]['deg_Mg_pH10'].values
ss['deg_Mg_pH10'] = x

In [27]:
x = l3_intercept
for i,col in enumerate(select_col3):
    x += l3_weights[i] * sub[col]['deg_Mg_50C'].values
ss['deg_Mg_50C'] = x

In [28]:
ss

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
id_seqpos,,,,,
id_00073f8be_0,0.792687,0.624065,0.0,0.536340,0.0
id_00073f8be_1,2.428278,3.237135,0.0,3.218790,0.0
id_00073f8be_2,1.714191,0.624565,0.0,0.684638,0.0
id_00073f8be_3,1.318104,1.154631,0.0,1.666085,0.0
id_00073f8be_4,0.766394,0.574457,0.0,0.829680,0.0
...,...,...,...,...,...
id_ffda94f24_125,1.213479,-0.059627,0.0,0.256124,0.0
id_ffda94f24_126,1.232491,0.100564,0.0,0.538518,0.0
id_ffda94f24_127,1.307839,0.062769,0.0,0.597884,0.0


In [29]:
ss.to_csv("submission.csv",index=True)

In [30]:
select_col1

[0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 16, 17, 18, 24, 25, 26, 30, 31, 32]

In [31]:
select_col2

[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 14, 16, 17, 18, 19, 22, 25, 26, 28, 30, 31, 32]

In [32]:
select_col3

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 19, 22, 23, 27, 28, 29, 30]